In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from sklearn import metrics

## Load input data

In [2]:
#Hardcoded values
timestep = 16 # 2 seconds
epoch = 200

In [3]:
path = 'MovementAAL/dataset/MovementAAL_RSS_'

sequences = list()
file_targets = pd.read_csv('MovementAAL/dataset/MovementAAL_target.csv')

targets = list()

for i in range(1,315):
    file_path = path + str(i) + '.csv'
    file = pd.read_csv(file_path, header=0)
    for j in range(0, (len(file) - timestep)+1):
        arr = file[j:j+timestep]
        sequences.append(arr)
        targets.append(file_targets.values[:,1][i-1])


In [4]:
print(len(sequences))
print(len(targets))
sequences[0]

8487
8487


,RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.904760,-0.48,0.285710,0.30
1,-0.571430,-0.32,0.142860,0.30
2,-0.380950,-0.28,-0.142860,0.35
3,-0.285710,-0.20,-0.476190,0.35
4,-0.142860,-0.20,0.142860,-0.20
5,-0.142860,-0.20,0.047619,0.00
6,-0.142860,-0.16,-0.380950,0.20
7,-0.142860,-0.04,-0.619050,-0.20
8,-0.095238,-0.08,0.142860,-0.55
9,-0.047619,0.04,-0.095238,0.05


In [5]:
groups = pd.read_csv('MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

In [6]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()


count    8487.0
mean       16.0
std         0.0
min        16.0
25%        16.0
50%        16.0
75%        16.0
max        16.0
dtype: float64

In [7]:
X_train, X_rest, Y_train, Y_rest = train_test_split(sequences, targets, test_size=0.5, random_state=0)
X_test, X_validation, Y_test, Y_validation  = train_test_split(X_rest, Y_rest, test_size=0.4, random_state=0)

print(len(X_train))
print(len(X_validation))
print(len(X_test))
print(len(X_train)+len(X_validation)+len(X_test))
print(len(sequences))

4243
1698
2546
8487
8487


In [8]:
X_train = np.array(X_train)
X_validation = np.array(X_validation)
X_test = np.array(X_test)

Y_train = np.array(Y_train)
Y_train = (Y_train+1)/2

Y_validation = np.array(Y_validation)
Y_validation = (Y_validation+1)/2

Y_test = np.array(Y_test)
Y_test = (Y_test+1)/2


In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(timestep, 4),dropout=0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               267264    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 267,521
Trainable params: 267,521
Non-trainable params: 0
_________________________________________________________________


In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=epoch, batch_size=256, callbacks=[chk], validation_data=(X_validation,Y_validation))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epoch)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model = load_model('models/best_model.pkl')
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

test_preds = (model.predict(X_test) > 0.5).astype("int32")

classification_report = metrics.classification_report(Y_test, test_preds)
print(classification_report)